In [1]:
# script to create store dimension landing

In [2]:
# Import required libraries
import sys
from lib.spark_session import get_spark_session
from lib.utils import date_data, get_string_cols, get_rundate
from lib.job_control import insert_log, get_max_timestamp
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp, lit
from datetime import datetime
from delta import DeltaTable
from lib.aws_s3 import archive_landing_object

In [3]:
# JOB Parameters
rundate = get_rundate()
schema_name = "edw_ld"
table_name = "dim_store_ld"
table_full_name = f"{schema_name}.{table_name}" 
landing_file_name = f"store_{rundate}.csv"
landing_file_path = f"s3a://datasatish/dw-with-pyspark/landing/store/{landing_file_name}"
print("SPARK_APP: JOB triggered for rundate - " + rundate)

SPARK_APP: JOB triggered for rundate - 20220101


In [4]:
# Generate Spark Session
spark: SparkSession = get_spark_session(f"Landing load - {table_full_name}")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

SPARK_APP: Spark UI - http://16804892cba9:4040


In [5]:
# Spark Configs
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.sql.parquet.mergeSchema", True)

In [6]:
# Create Raw dataframe and get col info
df_raw = spark \
    .read \
    .format("csv") \
    .option("header", True) \
    .load(landing_file_path)

print("SPARK_APP: Printing Raw Schema --")
df_raw.printSchema()

# Get Landing count
print("SPARK_APP: Landing data count - " + str(df_raw.count()))

SPARK_APP: Printing Raw Schema --
root
 |-- store_id: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)

SPARK_APP: Landing data count - 7


In [7]:
# Cast all columns to String
df_casted = df_raw.selectExpr(get_string_cols(spark, df_raw))
print("SPARK_APP: Casted all columns to String")

# Add audit columns
df_ld = df_casted.withColumn("insert_dt", current_timestamp()) \
    .withColumn("rundate", lit(rundate))
print("SPARK_APP: Added AUDIT column")

# Get Final Layer count
print("SPARK_APP: Final layer data count - " + str(df_ld.count()))
print("SPARK_APP: Printing Landing Schema --")
df_ld.printSchema()

SPARK_APP: Casted all columns to String
SPARK_APP: Added AUDIT column
SPARK_APP: Final layer data count - 7
SPARK_APP: Printing Landing Schema --
root
 |-- store_id: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- insert_dt: timestamp (nullable = false)
 |-- rundate: string (nullable = false)



In [8]:
# Write the data to landing layer checking if table exists
if get_max_timestamp(spark, schema_name, table_name) != "1900-01-01 00:00:00.000000":
    df_ld.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(table_full_name)
else:
    df_ld.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(table_full_name)

print("SPARK_APP: Data written to landing layer")

SPARK_APP: Data written to landing layer


In [9]:
# Archive the landing file
# (UPDATE: Python version in docker as 3.7 is not supported.)
if archive_landing_object(landing_file_name, "store"):
    print(f"SPARK_APP: Landing file {landing_file_name} archived")
else:
    print(f"SPARK_APP: ERROR - Landing file {landing_file_name} not archived. Please Archive the file manually")

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Unable to locate credentials
SPARK_APP: ERROR - Landing file store_20220101.csv not archived. Please Archive the file manually


In [10]:
# Add job details in JOB CONTROL
insert_log(spark, schema_name, table_name, datetime.now(), rundate)
print("SPARK_APP: Update JOB Control Log")

SPARK_APP: Update JOB Control Log


In [11]:
spark.sql(f"select * from edw.job_control where table_name = '{table_name}' order by insert_dt desc limit 1").show(truncate=False)

+-----------+------------+--------------------------+--------+-------------------------+
|schema_name|table_name  |max_timestamp             |rundate |insert_dt                |
+-----------+------------+--------------------------+--------+-------------------------+
|edw_ld     |dim_store_ld|2025-12-28 17:56:12.375593|20220101|2025-12-28 17:56:17.96417|
+-----------+------------+--------------------------+--------+-------------------------+



In [12]:
# Get the logs from delta table version
dt = DeltaTable.forName(spark, table_full_name)
dt.history().limit(1).select("version","operationMetrics.executionTimeMs", 
                                 "operationMetrics.numTargetRowsInserted",
                                "operationMetrics.numTargetRowsUpdated",
                                "operationMetrics.numOutputRows").show(1, False)

+-------+---------------+---------------------+--------------------+-------------+
|version|executionTimeMs|numTargetRowsInserted|numTargetRowsUpdated|numOutputRows|
+-------+---------------+---------------------+--------------------+-------------+
|2      |null           |null                 |null                |7            |
+-------+---------------+---------------------+--------------------+-------------+



In [13]:
# Generate Symlink manifest for Athena Access
dt.generate("symlink_format_manifest")
print("SPARK_APP: Symlink Manifest file generated")

SPARK_APP: Symlink Manifest file generated


In [14]:
spark.sql("select * from edw_ld.dim_store_ld").show()

+--------+------------+--------------+-----------+-----+--------+--------------+--------------------+--------+
|store_id|  store_name|       address|       city|state|zip_code|  phone_number|           insert_dt| rundate|
+--------+------------+--------------+-----------+-----+--------+--------------+--------------------+--------+
|    S001|Pet House KA|   123 Main St|    Anytown|   KA|   12345|91-88929-88888|2025-12-28 17:38:...|20220101|
|    S002|Pet House MH|    456 Elm St|Anothertown|   MH|   67890|91-99999-99999|2025-12-28 17:38:...|20220101|
|    S003|Pet House TN|   789 Oak Ave|    Bigcity|   TN|    9876|91-77777-77777|2025-12-28 17:38:...|20220101|
|    S004|Pet House OR|321 Birch Blvd| Small Town|   OR|   76684|91-88822-00000|2025-12-28 17:38:...|20220101|
|    S005|Pet House WB|   654 Pine St|   Busytown|   WB|   11111|91-00002-22222|2025-12-28 17:38:...|20220101|
|    S006|Pet House JK|  987 Cedar Rd|  Hill Town|   JK|   22222|91-33330-33333|2025-12-28 17:38:...|20220101|
|

In [15]:
spark.stop()